In [1]:
import numpy as np
import os
import os.path
import pandas as pd
import torch
import xlrd
import openpyxl
from sklearn import preprocessing

In [10]:
def minmax(data) -> pd.DataFrame:
    label_column = data['label']

    normalized_data = data.drop(columns=['label'])

    scaler = preprocessing.MinMaxScaler()
    normalized_data = scaler.fit_transform(normalized_data)

    columns = data.columns[:-1]
    normalized_df = pd.DataFrame(normalized_data, columns=columns)

    normalized_df['label'] = label_column.values

    return normalized_df


In [11]:
import random

use_cols = ['TWE_set',
            'TEI', 'TWEI', 'TEO', 'TWEO', 'TCI', 'TWCI', 'TCO', 'TWCO',
            'TSI', 'TSO', 'TBI', 'TBO', 'Cond Tons', 'Cooling Tons', 'Shared Cond Tons',
            'Cond Energy Balance', 'Evap Tons', 'Shared Evap Tons', 'Building Tons',
            'Evap Energy Balance', 'kW', 'COP', 'kW/Ton', 'FWC', 'FWE', 'TEA', 'TCA',
            'TRE', 'PRE', 'TRC', 'PRC', 'TRC_sub', 'T_suc', 'Tsh_suc', 'TR_dis', 'Tsh_dis',
            'P_lift', 'Amps', 'RLA%', 'Heat Balance%', 'Tolerance%',
            'Unit Status', 'Active Fault', 'TO_sump', 'TO_feed', 'PO_feed', 'PO_net',
            'TWCD', 'TWED', 'VSS', 'VSL', 'VH', 'VM', 'VC', 'VE', 'VW', 'TWI', 'TWO',
            'THI', 'THO', 'FWW', 'FWH', 'FWB'
            ]
normal = pd.read_excel('data_chiller/Benchmark Tests/normal.xls', usecols=use_cols)
fwc40 = pd.read_excel('data_chiller/Reduced condenser water flow/fwc40.xls', usecols=use_cols)
fwe40 = pd.read_excel('data_chiller/Reduced evaporator water flow/fwe40.xls', usecols=use_cols)
rl40 = pd.read_excel('data_chiller/Refrigerant leak/rl40.xls', usecols=use_cols)
ro40 = pd.read_excel('data_chiller/Refrigerant overcharge/ro40.xls', usecols=use_cols)
eo50 = pd.read_excel('data_chiller/Excess oil/eo50.xls', usecols=use_cols)
cf45 = pd.read_excel('data_chiller/Condenser fouling/cf45.xls', usecols=use_cols)
nc5 = pd.read_excel('data_chiller/Non-condensables in refrigerant/nc5.xls', usecols=use_cols)

fault_data_all = [normal, fwc40, fwe40, rl40, ro40, eo50, cf45, nc5]
random.shuffle(fault_data_all)

for i, df in enumerate(fault_data_all):
    df['label'] = i

fault_data = pd.concat(fault_data_all, ignore_index=True)

s_fault_data = fault_data[(fault_data['TWE_set'] == 40)]
t_fault_data = fault_data[(fault_data['TWE_set'] == 50)]

In [12]:
s_fault_data = s_fault_data[s_fault_data['label'].isin([0, 1, 2, 3])]

In [13]:
t_fault_data['label'] = t_fault_data['label'].apply(lambda x: 4 if x not in [0, 1, 2, 3] else x)

C:\Users\hhz\AppData\Local\Temp\ipykernel_8760\3091150995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_fault_data['label'] = t_fault_data['label'].apply(lambda x: 4 if x not in [0, 1, 2, 3] else x)


In [14]:
s_fault_data.to_csv('data_chiller/ODA/source_40_obda.csv')
t_fault_data.to_csv('data_chiller/ODA/target_50_obda.csv')

In [19]:
s_fault_data

,TWE_set,TEI,TWEI,TEO,TWEO,TCI,TWCI,TCO,TWCO,TSI,...,VE,VW,TWI,TWO,THI,THO,FWW,FWH,FWB,label


In [9]:
#ODA
for domain in [40, 45, 50]:
    s_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_{}.csv'.format(domain))
    t_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_{}.csv'.format(domain))
    s_fault_data = s_fault_data[s_fault_data['label'].isin([0, 1, 2])]
    t_fault_data['label'] = t_fault_data['label'].apply(lambda x: 3 if x not in [0, 1, 2] else x)
    s_fault_data.to_csv('data_chiller/ODA/source_{}_obda.csv'.format(domain))
    t_fault_data.to_csv('data_chiller/ODA/target_{}_obda.csv'.format(domain))

In [10]:
s_fault_data = s_fault_data[s_fault_data['label'].isin([0, 1, 2, 3])]
t_fault_data['label'] = t_fault_data['label'].apply(lambda x: 4 if x not in [0, 1, 2, 3] else x)

# s_fault_data.to_csv('data_chiller/ODA/source_45_obda.csv')
t_fault_data.to_csv('data_chiller/ODA/target_45_obda.csv')

In [31]:
# PDA
s_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_40.csv')
t_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_50.csv')

t_fault_data = t_fault_data[t_fault_data['label'].isin([0, 1, 2, 3, 4, 5])]

s_fault_data.to_csv('data_chiller/PDA/source_40_pda.csv')
t_fault_data.to_csv('data_chiller/PDA/target_50_pda.csv')

In [67]:
#OPDA
s_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_50.csv')
t_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_45.csv')

In [68]:
s_fault_data = s_fault_data[s_fault_data['label'].isin([0, 1, 2, 3, 4, 5])]
t_fault_data['label'] = t_fault_data['label'].apply(lambda x: 6 if x not in [0, 1, 2] else x)

In [69]:
s_fault_data.to_csv('data_chiller/OPDA/source_50_opda.csv')
t_fault_data.to_csv('data_chiller/OPDA/target_45_opda.csv')

In [12]:
for domain in [40, 45, 50]:
    s_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_{}.csv'.format(domain))
    t_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_{}.csv'.format(domain))
    s_fault_data = s_fault_data[s_fault_data['label'].isin([0, 1, 2, 3, 4])]
    t_fault_data= t_fault_data[t_fault_data['label'].isin([0, 1, 2, 5, 6, 7])]
    t_fault_data['label'] = t_fault_data['label'].apply(lambda x: 5 if x not in [0, 1, 2] else x)
    # s_fault_data = s_fault_data[s_fault_data['label'].isin([0, 1, 2, 3, 4, 5])]
    # t_fault_data['label'] = t_fault_data['label'].apply(lambda x: 6 if x not in [0, 1, 2] else x)
    s_fault_data.to_csv('data_chiller/OPDA/source_{}_opda.csv'.format(domain))
    t_fault_data.to_csv('data_chiller/OPDA/target_{}_opda.csv'.format(domain))

In [53]:
#CDA
s_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_50.csv')
t_fault_data = pd.read_csv('data_chiller/data_res_feature/fault_data_40.csv')
s_fault_data.to_csv('data_chiller/CDA/source_50_cda.csv')
t_fault_data.to_csv('data_chiller/CDA/target_40_cda.csv')